# 1. linux 内核模块
linux 内核模块三要素：入口、出口、许可证

# 2. 字符设备驱动
按照字节流访问，只能顺序访问，不能无序访问的设备。

## 2.1 接口

In [ ]:
#include <linux/fs.h>
/**
 * @功能：注册字符设备驱动
 * @参数：major：主设备号，name：设备名，fops：文件操作集
 * @返回值：成功返回0，失败返回负值
 * @备注：如果major为0，则由系统自动分配主设备号
*/
int register_chrdev(unsigned int major, const char *name, const struct file_operations *fops)

/**
 * @功能：注销字符设备驱动
 * @参数：major：主设备号，name：设备名
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev(unsigned int major, const char *name)

In [ ]:
#include <linux/uaccess.h>
/**
 * @功能：从用户空间拷贝数据到内核空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

/**
 * @功能：从内核空间拷贝数据到用户空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

In [ ]:
#include <linux/io.h>
/**
 * @功能：将物理地址映射到内核虚拟地址空间
 * @参数：phy_addr：物理地址，size：映射字节数
 * @返回值：映射成功返回内核虚拟地址，失败返回NULL
 * @备注：无
*/
void  *ioremap(resource_size_t phy_addr, size_t size)

/**
 * @功能：解除内核虚拟地址空间的映射
 * @参数：virt_addr：内核虚拟地址
 * @返回值：无
 * @备注：无
*/
void iounmap(volatile void  *virt_addr)

In [ ]:
#include <linux/device.h>
/**
 * @功能：创建一个class
 * @参数：owner：模块的拥有者，name：class的名字
 * @返回值：成功返回class指针，失败返回NULL
 * @备注：无
*/
struct class * class_create(owner, name)

// void class_destroy(struct class *cls)
/**
 * @功能：创建一个device
 * @参数：class：class指针，parent：父设备指针，devt：设备号，drvdata：驱动数据，fmt：设备名，...：变参
 * @返回值：成功返回device指针，失败返回NULL
 * @备注：fmt,... - 设备名，如 "myled%d"，%d - myled0, myled1, ...
*/
struct device *device_create(struct class *class, struct device *parent, dev_t devt, void *drvdata, const char *fmt, ...)

In [ ]:
#include <linux/cdev.h>
/**
 * @功能：分配一个cdev对象
 * @参数：无
 * @返回值：成功返回cdev指针，失败返回NULL
 * @备注：无
*/
struct cdev *cdev_alloc(void)

/**
 * @功能：初始化一个cdev对象
 * @参数：cdev：cdev指针，fops：文件操作集
 * @返回值：无
 * @备注：无
*/
void cdev_init(struct cdev *cdev, const struct file_operations *fops)

/**
 * @功能：注册指定的设备号
 * @参数：from: 起始设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：主设备号不能超过511，
*/
register_chrdev_region(dev_t from, unsigned count, const char *name)
/**
 * @功能：注销指定的设备号
 * @参数：from: 起始设备号，count：设备号个数
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev_region(dev_t from, unsigned count)

/**
 * @功能: 动态注册设备号
 * @参数：dev：申请到的设备号指针，firstminor：起始次设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int alloc_chrdev_region(dev_t *dev, unsigned int firstminor, unsigned int count, const char *name)

/**
 * @功能：添加一个cdev对象到系统
 * @参数：p：cdev指针，dev：设备号，count：设备号个数
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int cdev_add(struct cdev *p, dev_t dev, unsigned count)

## 2.2 结构体

In [ ]:
// 文件操作结构体
// struct file_operations {
//     struct module *owner;
//     loff_t (*llseek) (struct file *, loff_t, int);
//     ssize_t (*read) (struct file *, char __user *, size_t, loff_t *);
//     ssize_t (*write) (struct file *, const char __user *, size_t, loff_t *);
//     ssize_t (*aio_read) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
//     ssize_t (*aio_write) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
//     int (*readdir) (struct file *, void *, filldir_t);
//     unsigned int (*poll) (struct file *, struct poll_table_struct *);
//     long (*unlocked_ioctl) (struct file *, unsigned int, unsigned long);
//     long (*compat_ioctl) (struct file *, unsigned int, unsigned long);
//     int (*mmap) (struct file *, struct vm_area_struct *);
//     int (*open) (struct inode *, struct file *);
//     int (*flush) (struct file *, fl_owner_t id);
//     int (*release) (struct inode *, struct file *);
//     int (*fsync) (struct file *, loff_t, loff_t, int datasync);
//     int (*aio_fsync) (struct kiocb *, int datasync);
//     int (*fasync) (int, struct file *, int);
//     int (*lock) (struct file *, int, struct file_lock *);
//     ssize_t (*sendpage) (struct file *, struct page *, int, size_t, loff_t *, int);
//     unsigned long (*get_unmapped_area)(struct file *, unsigned long, unsigned long, unsigned long, unsigned long);
//     int (*check_flags)(int);
//     int (*flock) (struct file *, int, struct file_lock *);
//     ssize_t (*splice_write)(struct pipe_inode_info *, struct file *, loff_t *, size_t, unsigned int);
//     ssize_t (*splice_read)(struct file *, loff_t *, struct pipe_inode_info *, size_t, unsigned int);
//     int (*setlease)(struct file *, long, struct file_lock **, void **);
//     long (*fallocate)(struct file *file, int mode, loff_t offset, loff_t len);
// };

In [ ]:
// 文件结构体
// struct file {
//     struct path     f_path;
//     struct inode    *f_inode;       /* cached value */
//     const struct file_operations    *f_op;
//     spinlock_t      f_lock;         /* f_ep_links, f_flags, no IRQ */
//     atomic_long_t   f_count;
//     unsigned int    f_flags;
//     fmode_t         f_mode;
//     loff_t          f_pos;
//     struct fown_struct f_owner;
//     const struct cred *f_cred;
//     struct file_ra_state    f_ra;
//     u64             f_version;
// #ifdef CONFIG_SECURITY
//     void            *f_security;
// #endif
//     /* needed for tty driver, and maybe others */
//     void            *private_data;
// #ifdef CONFIG_EPOLL
//     /* Used by fs/eventpoll.c to link all the hooks to this file */
//     struct list_head    f_ep_links;
//     struct list_head    f_tfile_llink;
// #endif /* #ifdef CONFIG_EPOLL */
//     struct address_space    *f_mapping;
// #ifdef CONFIG_DEBUG_WRITECOUNT
//     unsigned long f_mnt_write_state;
// #endif
// };

# 3. 内核中的并发和竞态
当多个进程通过同一个驱动（临界资源）访问硬件的时候，竞态就会产生。有 5 种解决方案来优化程序。

## 1.1 中断屏蔽
暂时的关闭中断，只针对单核处理器有效。

In [ ]:
local_irq_disable();
// 临界区
local_irq_enable();

## 1.2 自旋锁（忙等锁）
除首次获得锁之外的进程尝试获取锁会进入自旋状态直到获得锁的进程释放锁。

In [ ]:
// 由于自旋锁内的代码不能太长，所以常用来加锁一个信号量
int flag;
// 1. 定义一个自旋锁
spinlock_t my_lock;
// 2. 初始化自旋锁
spin_lock_init(&my_lock);
// 3. 加锁
spin_lock(&my_lock);
// 4. 解锁
spin_unlock(&my_lock);